In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = 32
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 100

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../input/devnagri-script-classification/Data/Train",
    shuffle = True,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names

In [ ]:
class_names

In [ ]:
for image_batch, label_batch in dataset.take(1):
    plt.imshow(image_batch[0].numpy().astype("uint8"))
    plt.title(class_names[label_batch[0]])
    plt.axis("off")

In [ ]:
def partition(data, train_split=0.9, valid_split=0.1, shuffle=True, shuffle_size=10000):
    data_size = len(data)
    
    if shuffle:
        data = data.shuffle(shuffle_size, seed=12)
    
    train_size = int(data_size*train_split)
    
    train_ds = data.take(train_size)
    valid_ds = data.skip(train_size)

    return train_ds, valid_ds

In [ ]:
train_ds, valid_ds = partition(dataset)

In [ ]:
len(train_ds), len(valid_ds)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) 
valid_ds = valid_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE) 

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
Data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1)
])

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(class_names)

model = tf.keras.models.Sequential([
    resize_and_rescale,
    Data_augmentation,
    
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(1, 1), padding='same', activation='relu', input_shape = input_shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2,  padding='same'),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2,  padding='same'),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2,  padding='same'),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(n_classes, activation='softmax')    
])

model.build(input_shape = input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=valid_ds,
    verbose=1
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("model")

In [ ]:
sample = pd.read_csv('../input/devnagri-script-classification/Data/sample_submission.csv')

In [ ]:
import os
import cv2

folder_path = '../input/devnagri-script-classification/Data/Test'

images = []

for i in range(sample.shape[0]):
    img = os.path.join(folder_path, str(i+1)+'.png')
    img = cv2.imread(img)
    images.append(img)

In [ ]:
images = np.array(images)
images.shape

In [ ]:
temp = np.argmax(model.predict(images), axis=1)
preds=[]
for i in temp:
    preds.append(class_names[i])

In [ ]:
sample['Category'] = preds
sample.to_csv('submission.csv', index=False)